### Install Dependencies


In [0]:
!pip install kaggle

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# Restart the Python Kernel
%restart_python

In [0]:
# Verify Kaggle CLI is Active
!kaggle --version

I0000 00:00:1767972969.702137    4343 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


Kaggle API 1.8.3


### Configure Kaggle Credentials

In [0]:
import os

os.environ["KAGGLE_USERNAME"] = ""
os.environ["KAGGLE_KEY"] = ""

print("✅ Kaggle credentials configured")


✅ Kaggle credentials configured


### Create Database Schema


In [0]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce
""")

DataFrame[]

### Create Volume for Data Storage

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_data
""")

DataFrame[]

### Download Dataset from Kaggle

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors
ecommerce-behavior-data-from-multi-category-store.zip: Skipping, found more recently modified local copy (use --force to force download)


### Extract Downloaded Dataset

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
unzip -o ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            
total 18G
-rwxrwxrwx 1 spark-f993ba66-279c-4126-8cc3-59 nogroup 8.4G Jan  9 15:37 2019-Nov.csv
-rwxrwxrwx 1 spark-f993ba66-279c-4126-8cc3-59 nogroup 5.3G Jan  9 15:39 2019-Oct.csv
-rwxrwxrwx 1 nobody                           nogroup 4.3G Jan  9 14:24 ecommerce-behavior-data-from-multi-category-store.zip


### Clean up Zip File

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
rm -f ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

total 14G
-rwxrwxrwx 1 spark-f993ba66-279c-4126-8cc3-59 nogroup 8.4G Jan  9 15:37 2019-Nov.csv
-rwxrwxrwx 1 spark-f993ba66-279c-4126-8cc3-59 nogroup 5.3G Jan  9 15:39 2019-Oct.csv


### Restart Python Environment

In [0]:
%restart_python


### Load November 2019 Data

In [0]:
df_n = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv")

### Load October 2019 Data

In [0]:
df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")

### Display Statistics and Schema for Dataset

In [0]:
print(f"October 2019 - Total Events: {df.count():,}")
print("\n" + "="*60)
print("SCHEMA:")
print("="*60)
df.printSchema()


October 2019 - Total Events: 42,448,765

SCHEMA:
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)



### Display Sample Data

In [0]:
print("\n" + "="*60)
print("SAMPLE DATA (First 5 rows):")
print("="*60)
display(df.limit(5))


SAMPLE DATA (First 5 rows):


_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8
event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,null,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,null,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
